In [ ]:
from qiskit import QuantumCircuit, execute, Aer, transpile, QuantumRegister, ClassicalRegister
from qiskit.circuit.library import CSwapGate
import numpy as np
from qiskit.providers.aer import QasmSimulator
import matplotlib.pyplot as plt
import fnmatch

from qiskit.providers.aer.noise import NoiseModel
from qiskit.providers.aer.noise.errors import pauli_error, depolarizing_error

#### Define the error model

In [ ]:
from qiskit import

In [ ]:
qiskit.__version__

In [ ]:
def get_noise(p_meas, p_gate):
    error_meas = pauli_error([('X', p_meas), ('I', 1 - p_meas)])
    error_gate1 = depolarizing_error(p_gate, 1)
    error_gate2 = error_gate1.tensor(error_gate1)

    noise_model = NoiseModel()
    #noise_model.add_all_qubit_quantum_error(error_meas, "measure")
    #noise_model.add_all_qubit_quantum_error(error_gate1, ["x"])
    noise_model.add_all_qubit_quantum_error(error_gate2, ["cx"])

    return noise_model

#### Amplitudes correspond to the states: $|\phi\rangle = A_1|00\rangle + A_2|01\rangle +A_3|10\rangle + A_4|11\rangle$

In [ ]:
s0 = [1, 0, 0, 0]
s1 = [np.sqrt(5 / 9), np.sqrt(4 / 9), 0, 0]
s2 = [np.sqrt(5 / 9), -np.sqrt(1 / 9), 0. + 1j * np.sqrt(1 / 3), 0]
s3 = [np.sqrt(5 / 9), -np.sqrt(1 / 9), 0. - 1j * np.sqrt(1 / 3), 0]
initial_states = [s0, s1, s2, s3]

#### Set up the circuit

In [8]:
# Set up the circuit
qc = QuantumCircuit()
anc = QuantumRegister(2, 'anc')
states = QuantumRegister(8, 'phi')
cr = ClassicalRegister(10, 'm')
qc.add_register(anc, states, cr)

# Set up initial states
for i, r in enumerate(initial_states):
    qc.initialize(r, [states[2*i], states[2*i+1]])

# Add the CSWAP and measure the ancillaries
qc.barrier()
qc.h(anc[0])
qc.h(anc[1])

qc.fredkin(anc[0], states[0], states[2])
qc.fredkin(anc[0], states[1], states[3])
qc.fredkin(anc[1], states[2], states[4])
qc.fredkin(anc[1], states[3], states[5])

qc.measure(anc[0], cr[0])
qc.measure(anc[1], cr[1])
qc.barrier()

# # Perform the two Bell measurements
qc.cx(states[0], states[2])
qc.cx(states[1], states[3])
qc.cx(states[4], states[6])
qc.cx(states[5], states[7])
qc.h(states[0])
qc.h(states[1])
qc.h(states[4])
qc.h(states[5])
qc.barrier()

for i in range(8):
    qc.measure(states[i], cr[i+2])

# Draw
qc.draw()

░ ┌───┐            ┌─┐   »
anc_0: ──────────────────────────────────────────────░─┤ H ├─■──■───────┤M├───»
                                                     ░ ├───┤ │  │       └╥┘┌─┐»
anc_1: ──────────────────────────────────────────────░─┤ H ├─┼──┼──■──■──╫─┤M├»
                  ┌──────────────────────┐           ░ └───┘ │  │  │  │  ║ └╥┘»
phi_0: ───────────┤0                     ├───────────░───────X──┼──┼──┼──╫──╫─»
                  │  initialize(1,0,0,0) │           ░       │  │  │  │  ║  ║ »
phi_1: ───────────┤1                     ├───────────░───────┼──X──┼──┼──╫──╫─»
            ┌─────┴──────────────────────┴─────┐     ░       │  │  │  │  ║  ║ »
phi_2: ─────┤0                                 ├─────░───────X──┼──X──┼──╫──╫─»
            │  initialize(0.74536,0.66667,0,0) │     ░          │  │  │  ║  ║ »
phi_3: ─────┤1                                 ├─────░──────────X──┼──X──╫──╫─»
        ┌───┴──────────────────────────────────┴───┐ ░             │  │  ║  ║ »
phi_4: ─┤0                                         ├─░─────────────X──┼──╫──╫─»
        │  initialize(0.74536,-0.33333,0.57735j,0) │ ░                │  ║  ║ »
phi_5: ─┤1                                         ├─░────────────────X──╫──╫─»
       ┌┴──────────────────────────────────────────┤ ░                   ║  ║ »
phi_6: ┤0                                          ├─░───────────────────╫──╫─»
       │  initialize(0.74536,-0.33333,-0.57735j,0) │ ░                   ║  ║ »
phi_7: ┤1                                          ├─░───────────────────╫──╫─»
       └───────────────────────────────────────────┘ ░                   ║  ║ »
 m: 10/══════════════════════════════════════════════════════════════════╩══╩═»
                                                                         0  1 »
«        ░                 ░                         
«anc_0: ─░─────────────────░─────────────────────────
«        ░                 ░                         
«anc_1: ─░─────────────────░─────────────────────────
«        ░      ┌───┐      ░ ┌─┐                     
«phi_0: ─░───■──┤ H ├──────░─┤M├─────────────────────
«        ░   │  └───┘┌───┐ ░ └╥┘┌─┐                  
«phi_1: ─░───┼────■──┤ H ├─░──╫─┤M├──────────────────
«        ░ ┌─┴─┐  │  └───┘ ░  ║ └╥┘┌─┐               
«phi_2: ─░─┤ X ├──┼────────░──╫──╫─┤M├───────────────
«        ░ └───┘┌─┴─┐      ░  ║  ║ └╥┘┌─┐            
«phi_3: ─░──────┤ X ├──────░──╫──╫──╫─┤M├────────────
«        ░      ├───┤      ░  ║  ║  ║ └╥┘┌─┐         
«phi_4: ─░───■──┤ H ├──────░──╫──╫──╫──╫─┤M├─────────
«        ░   │  └───┘┌───┐ ░  ║  ║  ║  ║ └╥┘┌─┐      
«phi_5: ─░───┼────■──┤ H ├─░──╫──╫──╫──╫──╫─┤M├──────
«        ░ ┌─┴─┐  │  └───┘ ░  ║  ║  ║  ║  ║ └╥┘┌─┐   
«phi_6: ─░─┤ X ├──┼────────░──╫──╫──╫──╫──╫──╫─┤M├───
«        ░ └───┘┌─┴─┐      ░  ║  ║  ║  ║  ║  ║ └╥┘┌─┐
«phi_7: ─░──────┤ X ├──────░──╫──╫──╫──╫──╫──╫──╫─┤M├
«        ░      └───┘      ░  ║  ║  ║  ║  ║  ║  ║ └╥┘
« m: 10/══════════════════════╩══╩══╩══╩══╩══╩══╩══╩═
«                             2  3  4  5  6  7  8  9

#### Define the function to run the simulation

In [ ]:
def run_simulation(measurement_noise, cx_noise, shots=20000, test=False):
    noise_model = get_noise(measurement_noise, cx_noise)

    # Run simulator
    simulator = QasmSimulator()
    compiled_circuit = transpile(qc, simulator)
    job = simulator.run(compiled_circuit, shots=shots, noise_model=noise_model)
    counts = job.result().get_counts(qc)

    # Print output and swap to top down ordering
    results = {}
    for k,v in counts.items():
        results[k[::-1]] = v/shots
    
    # Store the final states we are looking for
    antisymm = ["0101", "1101", "1010", "1110", "0111", "1011"]
    rs = {'r12': [f"00{a}????" for a in antisymm],
          'r13': [f"01{a}????" for a in antisymm],
          'r14': [f"11????{a}" for a in antisymm],
          'r23': [f"11{a}????" for a in antisymm],
          'r24': [f"01????{a}" for a in antisymm],
          'r34': [f"00????{a}" for a in antisymm]
         }
    overlaps = {}

    # Compute the overlaps
    for olap, combos in rs.items():
        p = 0
        for onec in combos:
            for res in fnmatch.filter(list(results.keys()), onec):
                p += results[res]
        overlaps[olap] = 1-8*p
    
    # Check the inequalities
    r12, r13, r14 = overlaps["r12"], overlaps["r13"], overlaps["r14"]
    r23, r24, r34 = overlaps["r23"], overlaps["r24"], overlaps["r34"]
    
    # Print the overlaps
    if test:
        print(f"r12 = {r12}")
        print(f"r13 = {r13}")
        print(f"r14 = {r14}")
        print(f"r23 = {r23}")
        print(f"r24 = {r24}")
        print(f"r34 = {r34}")

    #LHS face should be less than 1
    lhsface = r12 + r13 + r14 - r23 - r34 - r24
    return lhsface

#### Run a single simulation and test overlaps

In [ ]:
# True overlap values
print("Analytic")
print(f"r12 = {np.real(np.dot(np.conjugate(s0), s1) * np.dot(np.conjugate(s1), s0))}")
print(f"r13 = {np.real(np.dot(np.conjugate(s0), s2) * np.dot(np.conjugate(s2), s0))}")
print(f"r14 = {np.real(np.dot(np.conjugate(s0), s3) * np.dot(np.conjugate(s3), s0))}")
print(f"r23 = {np.real(np.dot(np.conjugate(s1), s2) * np.dot(np.conjugate(s2), s1))}")
print(f"r24 = {np.real(np.dot(np.conjugate(s1), s3) * np.dot(np.conjugate(s3), s1))}")
print(f"r34 = {np.real(np.dot(np.conjugate(s2), s3) * np.dot(np.conjugate(s3), s2))}")

print("Simulation")
ineq = run_simulation(0, 0, shots=20000, test=True)
print(f"\n\nineq = {ineq}")

#### Define a grid of error values and compute the coherence witness
Equal error in X and CX gates

In [ ]:
errgrid = np.arange(0, 1.001, 0.02)
inequality = [run_simulation(err, err, shots=20000) for err in errgrid]

In [ ]:
plt.plot(errgrid, inequality)
plt.axhline(1, color='k', ls='--')
plt.xlabel("CX Gate Error")
plt.ylabel("Coherence Witness")